### Hiva's functions

In [1]:
########################################################
#### Synonym and derived forms
########################################################
########################################################
# Synonym (copied 240619)
# required libraries:
import nltk 
from nltk.corpus import wordnet 
import requests

def get_synonyms(word):
    url = f"https://api.datamuse.com/words?rel_syn={word}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        synonyms = [entry['word'] for entry in data]
        return synonyms
    else:
        print("Error:", response.status_code)
        return []

########################################################
# related words (copied 240619)
def get_related_words(word, limit=60, topics=None):
    url = f"https://api.datamuse.com/words?ml={word}&max={limit}"
    
    if topics:
        url += f"&topics={topics}"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        related_words = [entry['word'] for entry in data]
        return related_words
    else:
        print("Error:", response.status_code)
        return []


########################################################
# A function to check if a word is in the list or not (copied 240619)    
def check_word_in_list(word, word_list):
    if word in word_list:
        return f"'{word}' is in the list."
    else:
        return f"'{word}' is not in the list."


In [5]:
import nltk
from nltk.stem import WordNetLemmatizer
from lemminflect import getInflection, getAllInflections    
getAllInflections('hide', upos='VERB')

{'VBD': ('hid',),
 'VBN': ('hidden',),
 'VBG': ('hiding',),
 'VBZ': ('hides',),
 'VB': ('hide',),
 'VBP': ('hide',)}

In [11]:
import lemminflect

word = 'hide'

# Get inflections for 'hide' as a verb
inflections = lemminflect.getAllInflections(word, upos='VERB')

# Add the base form 'hide' to the list of inflections
#inflections.append(word)

print("All inflections of 'hide' including base form:")
print(inflections)
type(inflections)


All inflections of 'hide' including base form:
{'VBD': ('hid',), 'VBN': ('hidden',), 'VBG': ('hiding',), 'VBZ': ('hides',), 'VB': ('hide',), 'VBP': ('hide',)}


dict

In [21]:
forms = list(inflections.values())

for item in forms:
    print(item[0], type(item[0]))

hid <class 'str'>
hidden <class 'str'>
hiding <class 'str'>
hides <class 'str'>
hide <class 'str'>
hide <class 'str'>


In [47]:
import lemminflect

def form_list(word):
    # Get all inflections for the word as a verb
    inflections = lemminflect.getAllInflections(word, upos='VERB')
    
    # Check if the word or any of its inflections are in the list of verb inflections
    forms = list(inflections.values())
    form_set = set()
    for item in forms:
        form_set.add(item[0])
    if form_set == {}:
        return word
    else:
        return list(form_set)

form_list('apple')
verb_forms_list

form_list('run')

['ran', 'run', 'running', 'runs']

In [33]:
inflections = lemminflect.getAllInflections('teacher', upos='VERB')
print(inflections)

{}


{'NNS': ('cats',), 'NN': ('cat',)}
('cat', 'cats')
{'NNS': ('children',), 'NN': ('child',)}
('child', 'childs')
{'NNS': ('mice',), 'NN': ('mouse',)}
('mouse', 'mouses')
{'NNS': ('books',), 'NN': ('book',)}
('book', 'books')
{}
('mice', 'mices')


('cat', 'cats')
('child', 'childs')
('mouse', 'mouses')
('book', 'book_of_account')
